In [7]:
%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import datetime
import math
import random

In [8]:
file_path = "./normal.csv"
df = pd.read_csv(file_path)
norvalue = df['value'].values

file_path = "./traindata.csv"
df = pd.read_csv(file_path)
convalue = df['value'].values

In [9]:
class Pid():
    def __init__(self, exp_val, con_val, kp, ki, kd):
        self.KP = kp
        self.KI = ki
        self.KD = kd
        self.exp_val = exp_val
        self.now_val = con_val
        self.sum_err = 0
        self.now_err = 0
        self.last_err = 0

    def cmd_pid(self):
        self.last_err = self.now_err
        self.now_err = self.exp_val - self.now_val
        self.sum_err += self.now_err
        # 这一块是严格按照公式来写的
        self.now_val = self.KP * (self.exp_val - self.now_val) \
                        + self.KI * self.sum_err + self.KD * (self.now_err - self.last_err)
        return self.now_val

In [10]:
def pid_soa(kpidi):
    lastvalue = []   #存储最后pid迭代的值
    for j in range(0,len(convalue)):   #一次控制2501个数据(即0.2s)，共用一个pid控制器
        my_Pid = Pid(norvalue[j], convalue[j], kpidi[0], kpidi[1], kpidi[2])   # nor正常  con异常  kp, ki, kd
        nowvalue = 0
        for i in range(1,3):  #每个数据迭代二次
            nowvalue = my_Pid.cmd_pid()  

        lastvalue.append(nowvalue)
                
    return np.linalg.norm(norvalue-lastvalue)

In [21]:
#种群初始化
Umax=0.9600   #最大隶属度值
Umin=0.0111   #最小隶属度值
wmax=0.9      #权重最大值
wmin=0.1
Dim=3         #维数
SwarmSize=35  #粒子群规模
MaxIter=100   #最大迭代次数
MinFit= 400   #最小适应值
Ub=np.array([0.5, 0.5, 0.5])    #PID参数的上限
Lb=np.array([-0.5,-0.5, -0.5])    #PID参数的下限
Indexfgbest=np.zeros((30,1))   #随机产生一个位置矩阵
Range=np.ones((SwarmSize,1))*(Ub-Lb)
Swarm=np.random.rand(SwarmSize,Dim)*Range+np.ones((SwarmSize,1))*Lb  #初始化粒子群 30x3矩阵
fSwarm=np.zeros((SwarmSize,1))   #计算粒子群适应值

for i in range(0,SwarmSize):
    fSwarm[i,:]=pid_soa(Swarm[i,:])    #粒子群的适应值，整个粒子群实参传给pidsoa，返回适应度值
    print(fSwarm[i],Swarm[i,:])

bestf=np.min(fSwarm)      #取最小适应值
bestin=np.argwhere(fSwarm==bestf)  #确定bestf在fSwarm矩阵的位置 ，得行和列的值
bestindex=bestin[0][0]      #取行索引
zbest=Swarm[bestindex,:]    #最佳Ki,Kd,Kp值
gbest=Swarm                 #所有Ki,Kd，Kp值
fgbest=fSwarm               #所有适应值
fzbest=bestf                #最佳适应值
# print(bestf,zbest,Swarm)
# print(fSwarm)

#迭代寻优
Di=0*np.random.rand(SwarmSize,Dim)
Buchang=0*np.random.rand(SwarmSize,Dim)
C=0*np.random.rand(SwarmSize,Dim)
Diego=0*np.random.rand(SwarmSize,Dim)
Dialt=0*np.random.rand(SwarmSize,Dim)
Dipro=0*np.random.rand(SwarmSize,Dim)
iter1=0
y_fitness=np.zeros((1,MaxIter))     #预先设置四个空矩阵
K_p=np.zeros((1,MaxIter))
K_i=np.zeros((1,MaxIter))
K_d=np.zeros((1,MaxIter))

[724.66711508] [-0.01084522  0.47786186  0.28084952]
[1141.62217925] [ 0.48566058  0.4431613  -0.4663163 ]
[993.13229246] [-0.26906698 -0.03186278  0.15688769]
[775.07483525] [-0.21609927  0.26858803 -0.10614379]
[987.24000133] [ 0.06473169  0.49425453 -0.22193366]
[1992.24250853] [-0.10683036 -0.46363492  0.32099097]
[1638.11660885] [ 0.38850444 -0.19333277  0.3521458 ]
[1930.04750883] [-0.07622772 -0.44354877  0.37686008]
[1121.0722531] [-0.29451013 -0.07564295  0.14134251]
[1314.56815379] [-0.03457877  0.48662046 -0.33060277]
[672.97669524] [ 0.31540653 -0.14345985 -0.1474514 ]
[1823.96435281] [ 0.16701288 -0.48245481  0.34229567]
[781.12923784] [ 0.23577703  0.27361935 -0.25280903]
[1509.51120291] [-0.34713315 -0.18859658  0.29824638]
[1990.12211303] [ 0.34778585 -0.19507732  0.47429618]
[902.13506185] [-0.29924014  0.2377178   0.45402518]
[1909.75690894] [-0.37271159 -0.20770162 -0.18525582]
[1557.98127385] [-0.29784511 -0.23054322  0.16407251]
[1269.35776399] [-0.42571522 -0.0626

In [22]:
while (iter1<MaxIter) and (fzbest>MinFit):
# for j in range(0,MaxIter):
    for i in range(0,SwarmSize):
        w=wmax-iter1*(wmax-wmin)/MaxIter   #权重
        Diego[i,:]=np.sign(gbest[i,:]-Swarm[i,:])          #确定利己方向  个体最佳位置-目前位置
        Dialt[i,:]=np.sign(zbest-Swarm[i,:])               #确定利他方向  全局最佳位置-目前位置
        if pid_soa(gbest[i,:]) >= pid_soa(Swarm[i,:]):    #确定预动方向  目前位置-之前位置
            Dipro[i,:]=-Di[i,:]
        else:
            Dipro[i,:]=Di[i,:]
            
        Di[i,:]=np.sign(w*Dipro[i,:]+0.5*Diego[i,:]+0.5*Dialt[i,:])  #搜索方向  0.5可变动
        #确定经验梯度方向
        Orderfgbest=-np.sort(-fgbest,axis=0)    #个体最佳适应降序排列，默认升序 写两个负号变为降序
        Indexfgbest=np.argsort(-fgbest,axis=0)   #确定初始值的位置
        u=Umax-(SwarmSize-(Indexfgbest[i,0]+1))*(Umax-Umin)/(SwarmSize-1)  #搜索者个体i当前位置适应度的隶属度
        U=u+(1-u)*random.random()
        H=(MaxIter-iter1)/MaxIter  #迭代中权重的变化
        C[i,:]=H*abs(zbest-10*np.random.rand(1,3))  #确定高斯函数的参数
        T=math.sqrt(-math.log(U))
        Buchang[i,:]=C[i,:]*T    #确定搜索步长
        for n in range(0,Dim):
            if Buchang[i,n]>(3*np.max(C[i,:])):    #大于三倍参数，就等于这个参数
                Buchang[i,n]=3*np.max(C[i,:])
                
        #更新位置
        Swarm[i,:]=Swarm[i,:]+Di[i,:]*Buchang[i,:]   #对个体位置进行更新（即种群进化或最小优化问题候选解更新）
        for k in range(0,Dim):
            if Swarm[i,k]>Ub[k]:
                Swarm[i,k]=Ub[k]
            if Swarm[i,k]<Lb[k]:
                Swarm[i,k]=Lb[k]
        #适应值
        fSwarm[i,:]=pid_soa(Swarm[i,:])     
        #个体最优更新
        if fSwarm[i]<fgbest[i]:
            gbest[i,:]=Swarm[i,:]
            fgbest[i]=fSwarm[i]
            
        #群体最优更新
        if fSwarm[i]<fzbest:
            zbest=Swarm[i,:]
            print(Swarm[i,:],fSwarm[i,0])
            fzbest=fSwarm[i,0]
            print(zbest,fzbest)

#             K_p[0,iter1-1] = zbest[0]
#             K_i[0,iter1-1] = zbest[1]
#             K_d[0,iter1-1] = zbest[2]
        

    iter1 = iter1 + 1
#     print(zbest,fzbest)
#     print(zbest,fSwarm[i,0])
#     y_fitness[0,iter1-1] = fSwarm[i,0]   #单值出来
#     K_p[0,iter1-1] = zbest[0]
#     K_i[0,iter1-1] = zbest[1]
#     K_d[0,iter1-1] = zbest[2]

[ 0.37506667  0.29499    -0.16124236] 383.94342863181686
[ 0.37506667  0.29499    -0.16124236] 383.94342863181686
